NLP
Необходимо реализовать биграммную модель языка и обучить ее.

Обязательный функционал:

    Предсказание вероятности входного предложения
    Предсказание наиболее вероятных пар ко входному слову
    Продолжение входной фразы словами до заданной длины
    Использование сглаживания Лапласа для поддержки невстреченных ранее слов

Бонусы:

    Использование более интересных подходов к сглаживанию
    Многоуровневая модель с вариативной длиной n-граммов
    Поддержка знаков препинания

# Создание БД по n - граммам

In [16]:
from tqdm import tqdm_notebook as tn

In [1]:
%%time
f = open('book_to_teach/Mertv_Dushi.txt','r')#нужно еще распарсить по предложениям
kniga = [line.strip().split() for line in f if line != '\n']
f.close()

Wall time: 31.1 ms


In [2]:
def slianie(array):
    s = ''
    for x in array:
        s += x+' '
    return s.strip()    

In [3]:
def create_key(new_massive_slov, N):
    if N == 1:
        return new_massive_slov[0:N]
    else:
        return slianie(new_massive_slov[0:N])

In [27]:
def gramm(line):
    global last_posled, last_key, slovar_gramm, N, step, next_gramm
    
    new_massive_slov = last_posled+line
    f = True
    while f:
        if N + abs(next_gramm - N -step) <= len(new_massive_slov):
            new_key = create_key(new_massive_slov, N)

            if slovar_gramm.get(last_key) == None:
                slovar_gramm[last_key] = {new_key:1}
            else:
                if slovar_gramm[last_key].get(new_key) == None:
                    slovar_gramm[last_key][new_key] = 1
                else:
                    slovar_gramm[last_key][new_key] += 1

            last_key = new_key
            if N == 1:
                new_massive_slov = new_massive_slov[N:]
            else:
                new_massive_slov = new_massive_slov[N-1:]
            #print(new_massive_slov)
        else:
            last_posled = new_massive_slov
            f = False
            

In [29]:
#%%time
N = 1 #длина первого шингла 2
step = 1 #шаг отступа от первого слова 1 
next_gramm = 1 #длина связи шингла 2

slovar_gramm = {}
last_posled = []
last_key = ''

for line in tn(kniga):#[0:10] - 0 ns
    gramm(line)

In [30]:
slovar_gramm

{'': {'Мёртвые': 1},
 'Мёртвые': {'души': 1},
 'души': {'Николай': 1,
  'купчую?': 1,
  'в': 1,
  'будут': 1,
  'дело': 1,
  'нужны': 1,
  'умершими,': 1,
  'получил': 1,
  'свое': 1,
  'его,': 1,
  'Плюшкина,': 1,
  'не': 1,
  'те,': 1,
  'и': 2,
  'выдумка': 1,
  'уж': 1,
  '—': 1,
  'над': 1,
  'сей': 1,
  'нашей': 1,
  'некоторым': 1,
  'еще': 1,
  'были': 1,
  'наравне': 1,
  'своей': 1,
  'зависит': 1,
  'с': 1},
 'Николай': {'Гоголь': 1},
 'Гоголь': {'МЕРТВЫЕ': 1},
 'МЕРТВЫЕ': {'ДУШИ': 1},
 'ДУШИ': {'Текст': 1},
 'Текст': {'печатается': 1},
 'печатается': {'по': 1, 'в': 2, 'текст': 2},
 'по': {'изданию:': 1,
  'всей': 10,
  'себе;': 1,
  'торговым': 1,
  'истертым': 1,
  'совету': 1,
  'обыкновению,': 5,
  'просьбе': 1,
  'складам': 2,
  'своим': 3,
  'мнению': 2,
  'деревянному': 1,
  'тюлю.': 1,
  'бесконечно': 1,
  'сахарной': 1,
  'моде,': 3,
  'сторонам,': 3,
  'особенным': 2,
  'русскому': 5,
  'столу': 2,
  'усам!': 1,
  'усам!»': 1,
  'его': 7,
  'вечеринкам': 1,
  'мере

# Чистая вероятность появления каждого последующего n-грамма +сглаживание

In [49]:
def celar_P(f):
    global clear_veroiatnost, slovar_gramm
    for x in slovar_gramm:
        clear_veroiatnost[x] = {}
        summa = 0
        for y in slovar_gramm[x]:
            summa += slovar_gramm[x][y]
        if f:
            summa += 1
        chast = 100 / summa    
        for y in slovar_gramm[x]:
            clear_veroiatnost[x][y] = round(chast*slovar_gramm[x][y],4)
        if f:
            clear_veroiatnost[x]['Nothing'] = round(chast*1,4)

In [55]:
clear_veroiatnost = {}
f = False#True#определяет/включение отключение сглаживания
celar_P(f)

In [56]:
clear_veroiatnost

{'': {'Мёртвые': 100.0},
 'Мёртвые': {'души': 100.0},
 'души': {'Николай': 3.5714,
  'купчую?': 3.5714,
  'в': 3.5714,
  'будут': 3.5714,
  'дело': 3.5714,
  'нужны': 3.5714,
  'умершими,': 3.5714,
  'получил': 3.5714,
  'свое': 3.5714,
  'его,': 3.5714,
  'Плюшкина,': 3.5714,
  'не': 3.5714,
  'те,': 3.5714,
  'и': 7.1429,
  'выдумка': 3.5714,
  'уж': 3.5714,
  '—': 3.5714,
  'над': 3.5714,
  'сей': 3.5714,
  'нашей': 3.5714,
  'некоторым': 3.5714,
  'еще': 3.5714,
  'были': 3.5714,
  'наравне': 3.5714,
  'своей': 3.5714,
  'зависит': 3.5714,
  'с': 3.5714},
 'Николай': {'Гоголь': 100.0},
 'Гоголь': {'МЕРТВЫЕ': 100.0},
 'МЕРТВЫЕ': {'ДУШИ': 100.0},
 'ДУШИ': {'Текст': 100.0},
 'Текст': {'печатается': 100.0},
 'печатается': {'по': 20.0, 'в': 40.0, 'текст': 40.0},
 'по': {'изданию:': 0.1957,
  'всей': 1.9569,
  'себе;': 0.1957,
  'торговым': 0.1957,
  'истертым': 0.1957,
  'совету': 0.1957,
  'обыкновению,': 0.9785,
  'просьбе': 0.1957,
  'складам': 0.3914,
  'своим': 0.5871,
  'мнению': 

# Обратная вероятность

In [61]:
def create_back():
    global back_ver,clear_veroiatnost
    for slovo in clear_veroiatnost:
        back_ver[slovo] = {}
        max_b = 0
        for y in clear_veroiatnost[slovo]:
            if max_b < clear_veroiatnost[slovo][y]:
                max_b = clear_veroiatnost[slovo][y]
                
            if back_ver.get(clear_veroiatnost[slovo][y]) == None:
                back_ver[slovo][clear_veroiatnost[slovo][y]] = [y]
            else:
                back_ver[slovo][clear_veroiatnost[slovo][y]].append(y)
        back_ver[slovo]['Mmax'] = max_b

In [62]:
back_ver = {}
create_back()

In [63]:
back_ver

{'': {100.0: ['Мёртвые'], 'Mmax': 100.0},
 'Мёртвые': {100.0: ['души'], 'Mmax': 100.0},
 'души': {3.5714: ['с'], 7.1429: ['и'], 'Mmax': 7.1429},
 'Николай': {100.0: ['Гоголь'], 'Mmax': 100.0},
 'Гоголь': {100.0: ['МЕРТВЫЕ'], 'Mmax': 100.0},
 'МЕРТВЫЕ': {100.0: ['ДУШИ'], 'Mmax': 100.0},
 'ДУШИ': {100.0: ['Текст'], 'Mmax': 100.0},
 'Текст': {100.0: ['печатается'], 'Mmax': 100.0},
 'печатается': {20.0: ['по'], 40.0: ['текст'], 'Mmax': 40.0},
 'по': {0.1957: ['тяглам.'],
  1.9569: ['всей'],
  0.9785: ['обеим'],
  0.3914: ['бумагам,'],
  0.5871: ['делу'],
  1.3699: ['выражению'],
  0.7828: ['причине'],
  1.7613: ['всем'],
  5.8708: ['крайней'],
  1.1742: ['своей'],
  'Mmax': 5.8708},
 'изданию:': {100.0: ['Н.'], 'Mmax': 100.0},
 'Н.': {100.0: ['В.'], 'Mmax': 100.0},
 'В.': {5.2632: ['Гоголь,'],
  42.1053: ['Гоголя.)'],
  10.5263: ['Гоголя»,'],
  'Mmax': 42.1053},
 'Гоголь,': {100.0: ['Собрание'], 'Mmax': 100.0},
 'Собрание': {100.0: ['сочинений'], 'Mmax': 100.0},
 'сочинений': {100.0: ['в']

# Наиболее вероятная пара к входному слову

In [64]:
def para(slovo):
    global back_ver
    if back_ver.get(slovo) != None:
        return back_ver[slovo][back_ver[slovo]['Mmax']]
    else:
        return 'Нет такого слова'

In [74]:
def para_P(slovo):
    global back_ver
    if back_ver.get(slovo) != None:
        print('Mmax - лишь максимально возможная вероятность следующего слова')
        return back_ver[slovo]
    else:
        return 'Нет такого слова'

In [75]:
para('по')

['крайней']

In [76]:
para_P('по')

Mmax - лишь максимально возможная вероятность следующего слова


{0.1957: ['тяглам.'],
 1.9569: ['всей'],
 0.9785: ['обеим'],
 0.3914: ['бумагам,'],
 0.5871: ['делу'],
 1.3699: ['выражению'],
 0.7828: ['причине'],
 1.7613: ['всем'],
 5.8708: ['крайней'],
 1.1742: ['своей'],
 'Mmax': 5.8708}

# Продолжение входной фразы словами до заданной длины

In [89]:
def leng_phrase(slovo,N):
    global back_ver
    slovo += ' '
    new_slovo = slovo.strip()
    for x in range(N):
        if back_ver.get(new_slovo) == None:
            new_slovo = ''
            slovo += 'Невозможно продолжить! '
        else:
            new_slovo = back_ver[new_slovo][back_ver[new_slovo]['Mmax']][0]
            #print(new_slovo)
            slovo += new_slovo +' '
    return slovo.strip()

In [90]:
leng_phrase('по',4)

'по крайней мере того как'